In [35]:
from sklearn import svm
from sklearn.metrics import mean_squared_error
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import mean_squared_error, make_scorer
from hyperopt import rand, tpe, fmin, hp, Trials
import time

In [17]:
xtreino = np.load("Xtreino5.npy")
ytreino = np.load("ytreino5.npy")
xtest = np.load("Xteste5.npy")
ytest = np.load("yteste5.npy")

clf = svm.SVR(kernel="rbf")

<hr>

<h2>Intervalo dos hiperparâmetros:</h2><br>

<ul>
    <li>C entre $2^{-5}$ e $2^{15}$ (uniforme nos expoentes)</li>
    <li>gamma entre $2^{-15}$ e $2^3$ (uniforme nos expoentes)</li>
    <li>epsilon entre 0.05 a 1.0 (uniforme neste intervalo)</li>
</ul>

<hr>

<h3>1 - Random search</h3>

In [18]:
n_iters = 125
param_distributions = {"C":np.random.uniform(2**-5,2**15,n_iters),
                       "gamma":np.random.uniform(2**-15,2**3,n_iters),
                       "epsilon":np.random.uniform(10**-2,1,n_iters)}

score = make_scorer(mean_squared_error)

t0 = time.time()

svr = RandomizedSearchCV(clf,param_distributions , n_iter=n_iters, scoring=score, cv=5, iid=True)
svr.fit(xtreino, ytreino)
error = mean_squared_error(svr.predict(xtest), ytest)

svr_fit_time = time.time() - t0

In [14]:
print("SVR complexity and bandwidth selected and model fitted in %f s\n" % svr_fit_time)
print("Parameters: {0}".format(svr.cv_results_['params'][0]))
print("Mean squared error: %f" % (error))

SVR complexity and bandwidth selected and model fitted in 6.315539 s

Parameters: {'gamma': 4.042537423268548, 'epsilon': 0.4568485267850073, 'C': 9619.36113936479}
Mean squared error: 67.145633


<hr>
<br>
<h3>2 - Grid search</h3>
<hr>

In [22]:

n_iters = 5

param_distributions = {"C":np.random.uniform(2**-5,2**15,n_iters),
                       "gamma":np.random.uniform(2**-15,2**3,n_iters),
                       "epsilon":np.random.uniform(10**-2,1,n_iters)}


score = make_scorer(mean_squared_error)

svr =  GridSearchCV(clf,param_distributions, scoring=score, iid=True, cv=5)

svr.fit(xtreino, ytreino)
error = mean_squared_error(svr.predict(xtest), ytest)

svr_fit_time = time.time() - t0

In [23]:
print("SVR complexity and bandwidth selected and model fitted in %f s\n" % svr_fit_time)
print("Parameters: {0}".format(svr.cv_results_['params'][0]))
print("Mean squared error: %f" % (error))

SVR complexity and bandwidth selected and model fitted in 146.364886 s

Parameters: {'C': 22143.91274482454, 'epsilon': 0.029571976046935844, 'gamma': 7.596698063392936}
Mean squared error: 67.132163


<hr>
<br>
<h3>3 - Otimização bayesiana</h3>
<hr>

In [73]:
def objective(args):
    gamma = args[0][1]
    C = args[1][1]
    epsilon = args[2][1]
    clf = svm.SVR(kernel="rbf",gamma=gamma,C=C,epsilon=epsilon)
    clf.fit(xtreino,ytreino)
    return mean_squared_error(svr.predict(xtest), ytest)

space = [('gamma', hp.uniform('gamma', 2**-15,2**3)),
        ('C', hp.uniform('C', 2**-5,2**15)),
        ('epsilon', hp.uniform('epsilon', 10**-2,1))]

In [81]:
tpe_algo = tpe.suggest

tpe_trials = Trials()
tpe_best = fmin(fn=objective, space=space, algo=tpe_algo, trials=tpe_trials, 
                max_evals=125, rstate= np.random.RandomState(50))

100%|██████████| 125/125 [00:01<00:00, 61.38it/s, best loss: 67.13216262297617]


In [82]:
print(tpe_best)

{'C': 19612.051234748964, 'epsilon': 0.5886997301923083, 'gamma': 7.9693710974198595}


<hr>
<br>
<h3>4 - PSO</h3>
<hr>